## Batting Analysis

### Orange Cap 

In [1]:

import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

pio.renderers.default = 'notebook_connected'

# Load datasets
matches = pd.read_csv("../ipl_wrang/ingestion/cleaned_data/matches_cleaned.csv")
deliveries = pd.read_csv("../ipl_wrang/ingestion/cleaned_data/deliveries_cleaned.csv")
ipl_runs = pd.read_csv("../ipl_wrang/data/IPL_last_year_runs.csv")
ipl_wickets = pd.read_csv("../ipl_wrang/data/IPL_last_year_wickets.csv")

In [2]:

top_scorers = ipl_runs[['Player', 'Runs']].sort_values(by='Runs', ascending=False).head(10)

fig = px.bar(top_scorers, x='Runs', y='Player', orientation='h',
             title="Top 10 Run Scorers", text_auto=True)
fig.update_layout(yaxis={'categoryorder':'total ascending'}, xaxis_title="Runs", yaxis_title="Batsman")
fig.show()

### Best Strike Rate

In [3]:

batter_stats = deliveries.groupby('batter').agg({'batsman_runs':'sum', 'ball':'count'}).reset_index()
batter_stats['strike_rate'] = (batter_stats['batsman_runs'] / batter_stats['ball']) * 100
batter_stats = batter_stats[batter_stats['ball'] >= 300].sort_values(by='strike_rate', ascending=False).head(10)

fig = px.scatter(batter_stats, x="ball", y="strike_rate", size="batsman_runs",
                 hover_name="batter", title="Top Strike Rates (min 300 balls faced)")
fig.show()


### Spider chart

In [4]:
import pandas as pd
import plotly.graph_objects as go
import plotly.io as pio

pio.renderers.default = 'notebook_connected'

players = ['V Kohli', 'RG Sharma', 'SK Raina', 'DA Warner', 'CH Gayle']

filtered = deliveries[deliveries['batter'].isin(players)]

batter_stats = filtered.groupby('batter').agg({
    'batsman_runs': 'sum',
    'match_id': 'nunique'
}).reset_index()

batter_stats['balls_faced'] = filtered.groupby('batter').size().values
batter_stats['strike_rate'] = (batter_stats['batsman_runs'] / batter_stats['balls_faced']) * 100
batter_stats['avg_runs_per_match'] = batter_stats['batsman_runs'] / batter_stats['match_id']


fours = filtered[filtered['batsman_runs'] == 4].groupby('batter').size()
batter_stats['fours'] = batter_stats['batter'].map(fours).fillna(0).astype(int)


sixes = filtered[filtered['batsman_runs'] == 6].groupby('batter').size()
batter_stats['sixes'] = batter_stats['batter'].map(sixes).fillna(0).astype(int)


hundreds = (
    filtered.groupby(['batter', 'match_id'])['batsman_runs'].sum()
    .reset_index()
    .query('batsman_runs >= 100')
    .groupby('batter')
    .size()
)
batter_stats['hundreds'] = batter_stats['batter'].map(hundreds).fillna(0).astype(int)


batter_stats['norm_runs'] = batter_stats['batsman_runs'] / batter_stats['batsman_runs'].max() * 100
batter_stats['norm_strike_rate'] = batter_stats['strike_rate'] / batter_stats['strike_rate'].max() * 100
batter_stats['norm_avg_runs'] = batter_stats['avg_runs_per_match'] / batter_stats['avg_runs_per_match'].max() * 100
batter_stats['norm_fours'] = batter_stats['fours'] / batter_stats['fours'].max() * 100
batter_stats['norm_sixes'] = batter_stats['sixes'] / batter_stats['sixes'].max() * 100
batter_stats['norm_hundreds'] = batter_stats['hundreds'] / batter_stats['hundreds'].max() * 100


fig = go.Figure()

for i in range(len(batter_stats)):
    fig.add_trace(go.Scatterpolar(
        r=[
            batter_stats.loc[i, 'norm_runs'],
            batter_stats.loc[i, 'norm_strike_rate'],
            batter_stats.loc[i, 'norm_avg_runs'],
            batter_stats.loc[i, 'norm_fours'],
            batter_stats.loc[i, 'norm_sixes'],
            batter_stats.loc[i, 'norm_hundreds']
        ],
        theta=['Total Runs', 'Strike Rate', 'Avg Runs per Match', 'Fours', 'Sixes', 'Hundreds'],
        fill='toself',
        name=batter_stats.loc[i, 'batter']
    ))

fig.update_layout(
    polar=dict(
        radialaxis=dict(visible=True, range=[0, 100])
    ),
    title="Player Performance Radar Chart (with Fours, Sixes, Hundreds)",
    showlegend=True
)

fig.show()